In [1]:
import dask.array as da
import napari
import btrack
from tqdm.auto import tqdm
from skimage.io import imshow,  imsave
import matplotlib.pyplot as plt
import numpy as np
import itertools
import json
plt.rcParams["figure.figsize"] = (15,7)

### Load images from zarr and compress into max projections

In [2]:
gfp = da.from_zarr('/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/zarr/tzxy_stack_ch1.zarr/')
rfp = da.from_zarr('/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/zarr/tzxy_stack_ch2.zarr/')
gfp = da.sum(gfp, axis = 1, dtype='u2')
rfp = da.sum(rfp, axis = 1, dtype='u2')
masks = da.from_zarr(f'/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/zarr/tzxy_stack_ch99.zarr/',)# chunks = gfp.chunks)

In [3]:
masks

dask.array<from-zarr, shape=(75, 6048, 6048), dtype=uint16, chunksize=(1, 2160, 2160), chunktype=numpy.ndarray>

## Define features

In [4]:
FEATURES = [
  "area",
  "major_axis_length",
  "minor_axis_length",
  "orientation",
  "solidity",
  "mean_intensity",
]

In [5]:
%%time

objects = btrack.utils.segmentation_to_objects(
    masks, 
    gfp,
    properties = tuple(FEATURES), 
)

[INFO][2022/08/08 05:14:11 PM] Localizing objects from segmentation...
[INFO][2022/08/08 05:14:11 PM] Found intensity_image data
[INFO][2022/08/08 05:14:11 PM] Calculating weighted centroids using intensity_image
[INFO][2022/08/08 05:20:22 PM] Objects are of type: <class 'dict'>
[INFO][2022/08/08 05:20:23 PM] ...Found 34804 objects in 75 frames.


CPU times: user 18min 40s, sys: 35.4 s, total: 19min 15s
Wall time: 6min 12s


# Now look at changes hypothesis model parameters

In [6]:
search_radii = [100, 300, 600] ## how far to look for nearest linkable tracklet
seg_miss_rate = [0.1, 0.05, 0.01] ## proportion of cells that are missed by segmentation
lambda_time = [1, 5, 10] ## the larger this is, the longer the time period that can elapse before the probability of starting/ending a track is diminished
lamdda_dist = [1, 3, 10] ## the larger this is, the longer the distance that can elapse before the probability of starting/ending a track is diminished
lambda_link = [50, 100, 300] ## larger this is, more likely two tracklets are linked together (think needs to be upped for this)
#lambda_branch = [50] ## similar to before, but tracks that branch through a mitosis (n/a)
### probability for init to be calculated if dt,d within theta, 
### set relax to on to ignore, meaning that tracks can start anywhere (do not want to test this,)
theta_dist = [5, 20, 100] ###
theta_time = [1, 5, 25] ### 
### bin size for considering hypotheses
dist_thresh = [40, 80, 160] ## isotropic spatial 
time_thresh = [2, 4, 8] ## same for time
#apop_thresh = [100] ## n/a

In [7]:
param_dict_hypo_model = {'search_radii':search_radii, 
              'seg_miss_rate':seg_miss_rate,
              'lambda_time':lambda_time, 
              'lamdda_dist':lamdda_dist, 
              'lambda_link':lambda_link, 
              'theta_dist':theta_dist, 
              'theta_time':theta_time, 
              'dist_thresh':dist_thresh, 
              'time_thresh':time_thresh}

In [8]:
#print(list(itertools.product(*param_dict_hypo_model.values())))
print('Number of iterations to cover all params:',len(list(itertools.product(*param_dict_hypo_model.values()))))

Number of iterations to cover all params: 19683


This is obviously quite a few parameters to iterate over, so starting with the first three lambdas

In [9]:
param_dict_hypo_model = {#'search_radii':search_radii, 
              #'seg_miss_rate':seg_miss_rate,
              'lambda_time':lambda_time, 
              'lamdda_dist':lamdda_dist, 
              'lambda_link':lambda_link, 
              'theta_dist':theta_dist, 
              #'theta_time':theta_time, 
              'dist_thresh':dist_thresh, 
              #'time_thresh':time_thresh
}

In [10]:
#print(list(itertools.product(*param_dict_hypo_model.values())))
print('Number of iterations to cover all params:',len(list(itertools.product(*param_dict_hypo_model.values()))))

Number of iterations to cover all params: 243


In [12]:
import pickle

In [13]:
param_configs = list(itertools.product(*param_dict_hypo_model.values()))
stat_dict = {}
for config in tqdm(param_configs):
    try:
        ### assign values from config
        lambda_time_i = config[0]
        lamdda_dist_i = config[1]
        lambda_link_i = config[2]
        theta_dist_i = config[3]
        dist_thresh_i = config[4]
        ### load config file
        config_fn = '/home/dayn/analysis/BayesianTracker/models/param_test_config_hypo.json'
        with open(config_fn,'r+') as jsonfile:
            config_json = json.load(jsonfile)
            ### edit params
            config_json['TrackerConfig']['HypothesisModel']['lambda_time'] = lambda_time_i
            config_json['TrackerConfig']['HypothesisModel']['lamdda_dist_i'] = lamdda_dist_i
            config_json['TrackerConfig']['HypothesisModel']['lambda_link_i'] = lambda_link_i
            config_json['TrackerConfig']['HypothesisModel']['theta_dist_i'] = theta_dist_i
            config_json['TrackerConfig']['HypothesisModel']['dist_thresh_i'] = dist_thresh_i
            ### dno what this does lol
            jsonfile.seek(0)
            ### save out as temporary config
            json.dump(config_json, jsonfile, indent = 4)
            jsonfile.truncate()  

        ### now try tracking with that configuration
        # initialise a tracker session using a context manager
        with btrack.BayesianTracker() as tracker:
            # configure the tracker using a config file
            tracker.configure_from_file(config_fn)
            tracker.verbose = True
            tracker.max_search_radius = 500
            # use visual features to track
            tracker.features = FEATURES
            # append the objects to be tracked
            tracker.append(objects)
            # set the volume
            tracker.volume=((0, 6048), (0, 6048), (-1e5, 1e5))
            # track them (in interactive mode)
            tracker.track_interactive(step_size=25)
            # generate hypotheses and run the global optimizer
            #tracker.optimize()
            tracks = tracker.tracks
            # store the configuration
            cfg = tracker.configuration
            ## get lazy assessment of tracking performance
            average_track_len = np.mean([len(track) for track in tracks])
            ### store stats in dictionary
            stat_dict[average_track_len] = cfg
            print(config, average_track_len)
        with open('hypo_model_params_results_1.pkl', 'wb') as f:
            pickle.dump(stat_dict, f)
    except Exception as e: print(e)

  0%|          | 0/243 [00:00<?, ?it/s]

[INFO][2022/08/08 05:24:52 PM] Loaded btrack: /home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/btrack/libs/libtracker.so
[INFO][2022/08/08 05:24:52 PM] btrack (v0.4.5) library imported
[INFO][2022/08/08 05:24:52 PM] Starting BayesianTracker session
[INFO][2022/08/08 05:24:52 PM] Loading configuration file: /home/dayn/analysis/BayesianTracker/models/param_test_config_hypo.json
[INFO][2022/08/08 05:24:52 PM] Setting max_search_radius -> 500
[INFO][2022/08/08 05:24:52 PM] Setting features -> ['area', 'major_axis_length', 'minor_axis_length', 'orientation', 'solidity', 'mean_intensity']
[INFO][2022/08/08 05:24:52 PM] Objects are of type: <class 'list'>
[INFO][2022/08/08 05:24:52 PM] Setting volume -> ((0, 6048), (0, 6048), (-100000.0, 100000.0))
[INFO][2022/08/08 05:24:52 PM] Starting tracking... 
[INFO][2022/08/08 05:24:52 PM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2022/08/08 05:25:06 PM]  - Timing (Bayesian updates: 527.87ms, Linking: 12.08ms)
[INFO][2022/08/08 0

(1, 1, 50, 5, 40) 3.2823316150229425


[INFO][2022/08/08 05:25:19 PM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2022/08/08 05:25:33 PM]  - Timing (Bayesian updates: 518.89ms, Linking: 11.96ms)
[INFO][2022/08/08 05:25:33 PM]  - Probabilities (Link: 0.97446, Lost: 1.00000)
[INFO][2022/08/08 05:25:33 PM]  - Stats (Active: 2550, Lost: 51737, Conflicts resolved: 1956)
[INFO][2022/08/08 05:25:33 PM] Tracking objects in frames 25 to 49 (of 75)...
[INFO][2022/08/08 05:25:44 PM]  - Timing (Bayesian updates: 178.97ms, Linking: 5.49ms)
[INFO][2022/08/08 05:25:44 PM]  - Probabilities (Link: 0.94544, Lost: 1.00000)
[INFO][2022/08/08 05:25:44 PM]  - Stats (Active: 1721, Lost: 105261, Conflicts resolved: 3564)
[INFO][2022/08/08 05:25:44 PM] Tracking objects in frames 50 to 74 (of 75)...
[INFO][2022/08/08 05:25:45 PM]  - Timing (Bayesian updates: 11.74ms, Linking: 1.00ms)
[INFO][2022/08/08 05:25:45 PM]  - Probabilities (Link: 0.92668, Lost: 1.00000)
[INFO][2022/08/08 05:25:45 PM] SUCCESS.
[INFO][2022/08/08 05:25:45 PM]  - Found 

(1, 1, 50, 5, 80) 3.2823316150229425


...

(10, 10, 300, 100, 40) 3.2823316150229425


[INFO][2022/08/08 07:12:04 PM] Setting volume -> ((0, 6048), (0, 6048), (-100000.0, 100000.0))
[INFO][2022/08/08 07:12:04 PM] Starting tracking... 
[INFO][2022/08/08 07:12:04 PM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2022/08/08 07:12:17 PM]  - Timing (Bayesian updates: 516.77ms, Linking: 11.95ms)
[INFO][2022/08/08 07:12:17 PM]  - Probabilities (Link: 0.97446, Lost: 1.00000)
[INFO][2022/08/08 07:12:17 PM]  - Stats (Active: 2550, Lost: 51737, Conflicts resolved: 1956)
[INFO][2022/08/08 07:12:17 PM] Tracking objects in frames 25 to 49 (of 75)...
[INFO][2022/08/08 07:12:28 PM]  - Timing (Bayesian updates: 180.64ms, Linking: 6.68ms)
[INFO][2022/08/08 07:12:28 PM]  - Probabilities (Link: 0.94544, Lost: 1.00000)
[INFO][2022/08/08 07:12:28 PM]  - Stats (Active: 1721, Lost: 105261, Conflicts resolved: 3564)
[INFO][2022/08/08 07:12:28 PM] Tracking objects in frames 50 to 74 (of 75)...
[INFO][2022/08/08 07:12:29 PM]  - Timing (Bayesian updates: 11.81ms, Linking: 1.04ms)
[INFO][2022

(10, 10, 300, 100, 80) 3.2823316150229425


[INFO][2022/08/08 07:12:30 PM] Setting volume -> ((0, 6048), (0, 6048), (-100000.0, 100000.0))
[INFO][2022/08/08 07:12:30 PM] Starting tracking... 
[INFO][2022/08/08 07:12:30 PM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2022/08/08 07:12:44 PM]  - Timing (Bayesian updates: 518.36ms, Linking: 11.75ms)
[INFO][2022/08/08 07:12:44 PM]  - Probabilities (Link: 0.97446, Lost: 1.00000)
[INFO][2022/08/08 07:12:44 PM]  - Stats (Active: 2550, Lost: 51737, Conflicts resolved: 1956)
[INFO][2022/08/08 07:12:44 PM] Tracking objects in frames 25 to 49 (of 75)...
[INFO][2022/08/08 07:12:55 PM]  - Timing (Bayesian updates: 179.27ms, Linking: 6.41ms)
[INFO][2022/08/08 07:12:55 PM]  - Probabilities (Link: 0.94544, Lost: 1.00000)
[INFO][2022/08/08 07:12:55 PM]  - Stats (Active: 1721, Lost: 105261, Conflicts resolved: 3564)
[INFO][2022/08/08 07:12:55 PM] Tracking objects in frames 50 to 74 (of 75)...
[INFO][2022/08/08 07:12:56 PM]  - Timing (Bayesian updates: 11.88ms, Linking: 1.02ms)
[INFO][2022

(10, 10, 300, 100, 160) 3.2823316150229425


In [14]:
with open('hypo_model_params_results_final.pkl', 'wb') as f:
        pickle.dump(stat_dict, f)

# Testing 1307 combinations of tracking parameters

In [16]:
len(stat_dict)

1